Step 1: Install Required Packages

In [0]:

%pip install numpy==1.24.4 --no-cache-dir
%pip install --force-reinstall --no-deps langchain langchain-community langchain-openai
%pip install databricks-sdk
%pip install -U duckduckgo-search
%pip install langchain-core
%pip install langchain-text-splitters
%pip install aiohttp
dbutils.library.restartPython()



In [0]:
%pip install wikipedia
dbutils.library.restartPython()

In [0]:
from langchain.chat_models import ChatDatabricks
from langchain.agents import Tool, AgentExecutor, create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents.react.agent import create_react_agent
import random

llm = ChatDatabricks(
    endpoint="any databrick endpoint",
    temperature=0.7,
    max_tokens=1000
)

import os
os.environ["TAVILY_API_KEY"] = "Your Tavily API Key"
# Tools
wiki = WikipediaAPIWrapper()
#duck duck works well, if you search simultenously, it gives error
#search = DuckDuckGoSearchRun()
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()


def generate_joke(topic: str) -> str:
    prompt = f"Tell me a short, funny joke about {topic}."
    return llm([HumanMessage(content=prompt)])

tools = [
    Tool(name="Wikipedia", func=wiki.run, description="Get summary from Wikipedia"),
    Tool(name="YouTube Search", func=lambda q: search.run(f"{q} site:youtube.com"), description="Find YouTube videos"),
    Tool(name="Joke Generator", func=generate_joke,
    description="Generates a funny AI-generated joke about the given topic"
)
]

tool_names = [tool.name for tool in tools]

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful research assistant using ReAct framework.\n"
     "You have access to the following tools:\n\n{tools}\n\n"
     "When you answer, use the following format strictly:\n\n"
     "Question: the input question you must answer\n"
     "Thought: you should always think about what to do\n"
     "Action: the action to take, must be one of [{tool_names}]\n"
     "Action Input: the input to the action\n"
     "Observation: the result of the action\n"
     "...(Repeat Thought/Action/Action Input/Observation as needed)\n"
     "Final Answer: the final answer to the original question\n"),
    ("human", "{input}"),
    ("ai", "Thought: {agent_scratchpad}")
])
prompt.input_variables = ["input", "tool_names", "agent_scratchpad", "tools"]

# Agent & executor
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)
from langchain.schema.messages import AIMessage, HumanMessage, ToolMessage
agent_scratchpad = [
    AIMessage(content="I'll search Wikipedia."),
    ToolMessage(
        tool_call_id="tool-1",
        tool_name="Wikipedia",
        content="LangChain is a framework to build apps using LLMs."
    ),
    AIMessage(content=f"As requested, here's a joke about topic"),
    ToolMessage(
        tool_call_id="tool-2",
        tool_name="Joke Generator",
        content="Here's a joke about {topic}:"
    )
]
#agent_scratchpad.append(AIMessage(content="I'll search Wikipedia."))
#agent_scratchpad.append(ToolMessage(tool_call_id="tool-1",tool_name=wiki, content="LangChain is a framework to build apps using LLMs."))

# Function
def run_topic_agent(topic: str):
    query = f"""
    I want to learn about '{topic}'.
    1. Give me a summary from Wikipedia.
    2. Recommend a YouTube video to watch on this topic.
    3. End with a very funny joke about '{topic}'.
    """
    response = agent_executor.invoke({
        "input": query,
        "agent_scratchpad": agent_scratchpad,
        "tool_names": tool_names
    })
    print("\n📌 Final Response:\n", response["output"])

# Run it
run_topic_agent("Chiceken, leg piece")

Run Query